In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
class Model:
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    
    def _build_net(self):
        self.X  = tf.placeholder(tf.float32,shape=[None,784])
        self.X_img = tf.reshape(self.X,shape=[-1,28,28,1])
        self.Y = tf.placeholder(tf.float32,shape=[None,10])
        
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
            # convolution layer1
            W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01),name="weight1")
            L1 = tf.nn.conv2d(self.X_img,W1,strides=[1,1,1,1],padding='SAME')
            L1 = tf.nn.relu(L1)
            # pooling layer1
            L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L1 = tf.nn.dropout(L1,keep_prob=self.keep_prob) # return 14*14*32
            
            # convolution layer2
            W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01),name="weight2")
            L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
            L2 = tf.nn.relu(L2)
            # pooling layer2
            L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L2 = tf.nn.dropout(L2,keep_prob=self.keep_prob) # 7*7*64
            
            # convolution layer3
            W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01),name="weight3")
            L3 = tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
            L3 = tf.nn.relu(L3)
            # pooling layer3
            L3 = tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
            L3 = tf.nn.dropout(L3,keep_prob=self.keep_prob)
            # flatten L3 for DNN
            L3 = tf.reshape(L3,[-1,128*4*4])
            print(L3)
            # DNN
            W4 = tf.get_variable("W4",shape=(128*4*4,625),initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3,W4)+b4)
            L4 = tf.nn.dropout(L4,keep_prob=self.keep_prob)
            
            # Fully Conntec layer
            W5 = tf.get_variable("W5",shape=(625,10),initializer=tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            hypothesis = tf.matmul(L4,W5)+ b5
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        prediction = tf.argmax(hypothesis,1)
        is_correct = tf.equal(prediction,tf.argmax(self.Y,1))
        accuracy = tf.reduce_mean(tf.cast(is_correct,dtype=tf.float32))
        
    def predict(self,x_test,keep_prob=1.0):
        return sess(prediction,feed_dict={
            X : x_test,
            keep_prob : keep_prob
        })
    
    def get_accuracy(self, x_test, y_test, keep_prop=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})

    def train(self, x_data, y_data, keep_prop=0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})
            

In [3]:
# hyper parameter
learning_rate = 0.01
training_epoches = 15
batch_size = 100

with tf.Session() as sess:
    m1 = Model(sess,'m1')
    sess.run(tf.global_variables_initializer())
    
    print('Learning start!')
    
    for epoch in range(training_epoches):
        avg_cost = 0.0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = m1.train(batch_xs,batch_ys)
            avg_cost += cost_val / total_batch
        print('Epoch : ',epoch+1,"cost : ",avg_cost)
    print('End')

Tensor("m1/Reshape:0", shape=(?, 2048), dtype=float32)
Learning start!
Epoch :  1 cost :  0.492774507268
Epoch :  2 cost :  0.237309546457
Epoch :  3 cost :  0.22292546363
Epoch :  4 cost :  0.239927529029
Epoch :  5 cost :  0.216928871274
Epoch :  6 cost :  0.219050537917
Epoch :  7 cost :  0.224706224047
Epoch :  8 cost :  0.231244674745
Epoch :  9 cost :  0.220875707872
Epoch :  10 cost :  0.222913363799
Epoch :  11 cost :  0.224720434431


KeyboardInterrupt: 